In [1]:
import pandas as pd

clean_sampleset2000 = pd.read_csv('C:/Users/Devanand J/Desktop/Final/Yelp_dataset.csv')
restaurants_train2021 = pd.read_csv('C:/Users/Devanand J/Desktop/Final/SemEval.csv')

# Converting the 'overall_polarity' to 0 and 1 for compatibility 
clean_sampleset2000['overall_polarity'] = clean_sampleset2000['overall_polarity'].apply(lambda x: 0 if x == 1 else 1)


In [2]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    clean_sampleset2000['preprocessed_comments'], clean_sampleset2000['overall_polarity'], test_size=0.2, random_state=42)


In [3]:
from transformers import RobertaTokenizerFast

# Initializing the RoBERTa and tokenizing
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=512)


C:\Users\Devanand J\anaconda3\envs\gpt_cuda118\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels.tolist())
test_dataset = SentimentDataset(test_encodings, test_labels.tolist())


In [5]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

# Loading RoBERTa model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)


C:\Users\Devanand J\anaconda3\envs\gpt_cuda118\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=4,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    gradient_accumulation_steps=8,   # Accumulate gradients to simulate a larger batch size
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True                        # Enable mixed precision training
)


In [7]:
# Creating Trainer instance and training the model
trainer = Trainer(
    model=model,                         
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

trainer.train()


C:\Users\Devanand J\anaconda3\envs\gpt_cuda118\Lib\site-packages\accelerate\accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.686000,0.670046
2,0.274400,0.175717
3,0.180200,0.175756


Checkpoint destination directory ./results\checkpoint-50 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results\checkpoint-100 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results\checkpoint-150 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=150, training_loss=0.42456602096557616, metrics={'train_runtime': 1130.5577, 'train_samples_per_second': 4.246, 'train_steps_per_second': 0.133, 'total_flos': 1262933065728000.0, 'train_loss': 0.42456602096557616, 'epoch': 3.0})

In [8]:
# Saving the model and tokenizer for later use to load from last checkpoint
model.save_pretrained('./results/Roberta_sentiment_model')
tokenizer.save_pretrained('./results/Roberta_sentiment_model')


('./results/Roberta_sentiment_model\\tokenizer_config.json',
 './results/Roberta_sentiment_model\\special_tokens_map.json',
 './results/Roberta_sentiment_model\\vocab.json',
 './results/Roberta_sentiment_model\\merges.txt',
 './results/Roberta_sentiment_model\\added_tokens.json',
 './results/Roberta_sentiment_model\\tokenizer.json')

In [9]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Predict on test set
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)

# Calculate accuracy
accuracy = accuracy_score(test_labels, preds)
print(f"Accuracy: {accuracy}")

# Print classification report
report = classification_report(test_labels, preds, target_names=['negative', 'positive'])
print(report)


Evaluation results: {'eval_loss': 0.17575562000274658, 'eval_runtime': 11.6627, 'eval_samples_per_second': 34.297, 'eval_steps_per_second': 4.287, 'epoch': 3.0}
Accuracy: 0.945
              precision    recall  f1-score   support

    negative       0.92      0.98      0.95       201
    positive       0.97      0.91      0.94       199

    accuracy                           0.94       400
   macro avg       0.95      0.94      0.94       400
weighted avg       0.95      0.94      0.94       400



In [10]:
import torch
from tqdm import tqdm
import time

# Predict on the entire clean_sampleset2000 dataset with progress tracking
all_texts = clean_sampleset2000['preprocessed_comments'].tolist()
all_preds = []

batch_size = 16
num_batches = len(all_texts) // batch_size + (1 if len(all_texts) % batch_size != 0 else 0)

start_time = time.time()
for i in tqdm(range(num_batches), desc="Processing batches"):
    batch_texts = all_texts[i*batch_size:(i+1)*batch_size]
    batch_encodings = tokenizer(batch_texts, truncation=True, padding=True, max_length=512)
    batch_dataset = SentimentDataset(batch_encodings, [0]*len(batch_texts))  # Dummy labels

    batch_predictions = trainer.predict(batch_dataset)
    batch_preds = np.argmax(batch_predictions.predictions, axis=-1)
    all_preds.extend(batch_preds)

elapsed_time = time.time() - start_time
print(f"Processing completed in {elapsed_time:.2f} seconds")

# Add predictions to the dataframe
clean_sampleset2000['predicted_sentiment'] = all_preds

# Saveing the predictions
clean_sampleset2000.to_csv('C:/Users/Devanand J/Desktop/Final/Roberta_overallpredictions.csv', index=False)

Processing batches:   0%|          | 0/125 [00:00<?, ?it/s]

Processing batches:   1%|          | 1/125 [00:00<01:40,  1.23it/s]

Processing batches:   2%|▏         | 2/125 [00:01<01:08,  1.79it/s]

Processing batches:   2%|▏         | 3/125 [00:02<01:22,  1.49it/s]

Processing batches:   3%|▎         | 4/125 [00:02<01:12,  1.68it/s]

...

Processing batches:  98%|█████████▊| 123/125 [01:08<00:01,  1.64it/s]

Processing batches:  99%|█████████▉| 124/125 [01:09<00:00,  1.76it/s]

Processing batches: 100%|██████████| 125/125 [01:09<00:00,  1.79it/s]


Processing completed in 69.72 seconds


ModuleNotFoundError: No module named 'ace_tools'